In [1]:
import requests
import datetime
import time
import pandas as pd

In [1]:
page_id = "crazyck101"

app_id = "393833577650615"
app_secret = "3ab280dac70995e4d8e601cbc025b53a"

access_token = app_id + "|" + app_secret

In [2]:
# 判斷response有無正常 正常 200，若無隔五秒鐘之後再試
def request_until_succeed(url):
    success = False
    while success is False:
        try: 
            req = requests.get(url)
            if req.status_code == 200:
                success = True
        except Exception as e:
            print(e)
            time.sleep(5)
            print("Error for URL %s: %s" % (url, datetime.datetime.now()))
            print("Retrying.")

    return req

# 取得Facebook data
def getFacebookPageFeedData(page_id, access_token, num_statuses):

    # Construct the URL string; see http://stackoverflow.com/a/37239851 for
    # Reactions parameters
    base = "https://graph.facebook.com/v2.8"
    node = "/%s/posts" % page_id 
    fields = "/?fields=message,link,created_time,type,name,id," + \
            "comments.limit(0).summary(true),shares,reactions" + \
            ".limit(0).summary(true)"
    parameters = "&limit=%s&access_token=%s" % (num_statuses, access_token)
    url = base + node + fields + parameters

    # 取得data
    data = request_until_succeed(url).json()
    return data

# 取得該篇文章的 reactions like,love,wow,haha,sad,angry數目
def getReactionsForStatus(status_id, access_token):

    # See http://stackoverflow.com/a/37239851 for Reactions parameters
        # Reactions are only accessable at a single-post endpoint

    base = "https://graph.facebook.com/v2.6"
    node = "/%s" % status_id
    reactions = "/?fields=" \
            "reactions.type(LIKE).limit(0).summary(total_count).as(like)" \
            ",reactions.type(LOVE).limit(0).summary(total_count).as(love)" \
            ",reactions.type(WOW).limit(0).summary(total_count).as(wow)" \
            ",reactions.type(HAHA).limit(0).summary(total_count).as(haha)" \
            ",reactions.type(SAD).limit(0).summary(total_count).as(sad)" \
            ",reactions.type(ANGRY).limit(0).summary(total_count).as(angry)"
    parameters = "&access_token=%s" % access_token
    url = base + node + reactions + parameters

    # 取得data
    data = request_until_succeed(url).json()
    return data

def processFacebookPageFeedStatus(status, access_token):

    # 要去確認抓到的資料是否為空
    status_id = status['id']
    status_type = status['type']
    if 'message' not in status.keys():
        status_message = ''
    else:
        status_message = status['message']
    if 'name' not in status.keys():
        link_name = ''
    else:
        link_name = status['name']
    link = status_id.split('_')
    
    # 此連結可以回到該臉書上的post
    status_link = 'https://www.facebook.com/'+link[0]+'/posts/'+link[1]

    status_published = datetime.datetime.strptime(status['created_time'],'%Y-%m-%dT%H:%M:%S+0000')
    # 根據所在時區 TW +8
    status_published = status_published + datetime.timedelta(hours=8)
    status_published = status_published.strftime('%Y-%m-%d %H:%M:%S') 
    
    # 要去確認抓到的資料是否為空
    if 'reactions' not in status:
        num_reactions = 0
    else:
        num_reactions = status['reactions']['summary']['total_count']
    if 'comments' not in status:
        num_comments = 0
    else:
        num_comments = status['comments']['summary']['total_count']
    if 'shares' not in status:
        num_shares = 0
    else:
        num_shares = status['shares']['count']

    def get_num_total_reactions(reaction_type, reactions):
        if reaction_type not in reactions:
            return 0
        else:
            return reactions[reaction_type]['summary']['total_count']
    
    # 取得該篇文章的 reactions like,love,wow,haha,sad,angry數目
    reactions = getReactionsForStatus(status_id, access_token)
    
    num_loves = get_num_total_reactions('love', reactions)
    num_wows = get_num_total_reactions('wow', reactions)
    num_hahas = get_num_total_reactions('haha', reactions)
    num_sads = get_num_total_reactions('sad', reactions)
    num_angrys = get_num_total_reactions('angry', reactions)
    num_likes = get_num_total_reactions('like', reactions)

    # 回傳tuple形式的資料
    return (status_id, status_message, link_name, status_type, status_link,
            status_published, num_reactions, num_comments, num_shares,
            num_likes, num_loves, num_wows, num_hahas, num_sads, num_angrys)


def scrapeFacebookPageFeedStatus(page_id, access_token):
    # all_statuses 用來儲存的list,先放入欄位名稱
    all_statuses = [('status_id', 'status_message', 'link_name', 'status_type', 'status_link',
            'status_published', 'num_reactions', 'num_comments', 'num_shares',
            'num_likes', 'num_loves', 'num_wows', 'num_hahas', 'num_sads', 'num_angrys')]
    
    has_next_page = True 
    num_processed = 0   # 計算處理多少post
    scrape_starttime = datetime.datetime.now()

    print("Scraping %s Facebook Page: %s\n" % (page_id, scrape_starttime))

    statuses = getFacebookPageFeedData(page_id, access_token, 100)

    while has_next_page:
        for status in statuses['data']:

            # 確定有 reaction 再把結構化後的資料存入 all_statuses
            if 'reactions' in status:
                all_statuses.append(processFacebookPageFeedStatus(status,access_token))

            # 觀察爬取進度,每處理100篇post,就輸出時間,
            num_processed += 1
            if num_processed % 100 == 0:
                print("%s Statuses Processed: %s" % (num_processed, datetime.datetime.now()))

        # 每超過100個post就會有next,可以從next中取得下100篇, 直到沒有next
        if 'paging' in statuses.keys():
            statuses = request_until_succeed(statuses['paging']['next']).json()
        else:
            has_next_page = False

    print("\nDone!\n%s Statuses Processed in %s" % \
        (num_processed, datetime.datetime.now() - scrape_starttime))
    
    return all_statuses




In [4]:
all_statuses = scrapeFacebookPageFeedStatus(page_id, access_token)

Scraping crazyck101 Facebook Page: 2017-03-26 16:45:31.948302

100 Statuses Processed: 2017-03-26 16:46:01.338110
200 Statuses Processed: 2017-03-26 16:46:31.131566
300 Statuses Processed: 2017-03-26 16:47:01.993215
400 Statuses Processed: 2017-03-26 16:47:32.004315
500 Statuses Processed: 2017-03-26 16:48:02.507580

Done!
517 Statuses Processed in 0:02:36.667468


In [5]:
df = pd.DataFrame(all_statuses[1:], columns=all_statuses[0])

In [6]:

df.head()

status_id  \
0  1021581417961885_1231718840281474   
1  1021581417961885_1230806240372734   
2  1021581417961885_1230710647048960   
3  1021581417961885_1230681773718514   
4  1021581417961885_1229765993810092   

                                      status_message  \
0                                         趕稿中，明天晚上見喔   
1  糟糕 以後要說94bb惹 #狂阿編\n\n提名投稿狂新聞>> https://goo.gl/...   
2                                 禮拜一就要來抽獎囉！趕快分享不然掰掰   
3    殺得你措手不及 #狂阿編\n\n提名投稿狂新聞>> https://goo.gl/lrVe2h   
4  百戰百勝 嘿嘿嘿！ #狂阿編\n\n提名投稿狂新聞>> https://goo.gl/lrVe2h   

                                   link_name status_type  \
0                        68.media.tumblr.com        link   
1  鄉民小心了！法院認證「87」是公然侮辱 - 生活 - 自由時報電子報｜卡提諾狂新聞        link   
2                                     卡提諾狂新聞       video   
3                               三寶的逆襲｜卡提諾狂新聞        link   
4                              嚇到褲底ㄧ包｜卡提諾狂新聞        link   

                                         status_link     status_published  \
0  https://www.facebook.com/1021581417961885/post...  2017-03-25 17:12:34   
1  https://www.facebook.com/1021581417961885/post...  2017-03-24 21:42:19   
2  https://www.facebook.com/1021581417961885/post...  2017-03-24 19:43:14   
3  https://www.facebook.com/1021581417961885/post...  2017-03-24 18:59:00   
4  https://www.facebook.com/1021581417961885/post...  2017-03-23 21:27:45   

   num_reactions  num_comments  num_shares  num_likes  num_loves  num_wows  \
0          10419           366          73       8516         46        62   
1          12024           752        1184       8924         22       960   
2           2261           632           0       2161         19         9   
3           4327           141         156       3637         10       334   
4           2439            73          34       2024          5        90   

   num_hahas  num_sads  num_angrys  
0        957        23         815  
1       2021        36          61  
2         70         2           0  
3        300        11          35  
4         29         2         289

In [7]:
path = page_id+'_post.csv'
df.to_csv(path,index=False,encoding='utf8')

In [8]:
df

status_id  \
0    1021581417961885_1231718840281474   
1    1021581417961885_1230806240372734   
2    1021581417961885_1230710647048960   
3    1021581417961885_1230681773718514   
4    1021581417961885_1229765993810092   
5    1021581417961885_1229316033855088   
6    1021581417961885_1228690150584343   
7    1021581417961885_1228337190619639   
8    1021581417961885_1228223280631030   
9    1021581417961885_1228173080636050   
10   1021581417961885_1227361470717211   
11   1021581417961885_1226861697433855   
12   1021581417961885_1226432190810139   
13   1021581417961885_1226406717479353   
14   1021581417961885_1226278837492141   
15   1021581417961885_1226221827497842   
16   1021581417961885_1225501984236493   
17   1021581417961885_1225484724238219   
18   1021581417961885_1225318954254796   
19   1021581417961885_1224135674373124   
20   1021581417961885_1223387294447962   
21   1021581417961885_1223371937782831   
22   1021581417961885_1223331397786885   
23   1021581417961885_1223290171124341   
24   1021581417961885_1222523671200991   
25   1021581417961885_1222500434536648   
26   1021581417961885_1221873541266004   
27   1021581417961885_1221674141285944   
28   1021581417961885_1221554631297895   
29   1021581417961885_1220703274716364   
..                                 ...   
487  1021581417961885_1031489970304363   
488  1021581417961885_1031485223638171   
489  1021581417961885_1031375616982465   
490  1021581417961885_1031116203675073   
491  1021581417961885_1030813197038707   
492  1021581417961885_1029846297135397   
493  1021581417961885_1029774007142626   
494  1021581417961885_1029743570479003   
495  1021581417961885_1029687257151301   
496  1021581417961885_1029115717208455   
497  1021581417961885_1028857467234280   
498  1021581417961885_1028765440576816   
499  1021581417961885_1028733127246714   
500  1021581417961885_1027773560676004   
501  1021581417961885_1027711120682248   
502  1021581417961885_1024324787687548   
503  1021581417961885_1026705077449519   
504  1021581417961885_1026658750787485   
505  1021581417961885_1024936557626371   
506  1021581417961885_1024435981009762   
507  1021581417961885_1024435174343176   
508  1021581417961885_1024433291010031   
509  1021581417961885_1024424271010933   
510  1021581417961885_1024221367697890   
511  1021581417961885_1023519924434701   
512  1021581417961885_1023504487769578   
513  1021581417961885_1023469851106375   
514  1021581417961885_1023380531115307   
515  1021581417961885_1022699851183375   
516  1021581417961885_1022561054530588   

                                        status_message  \
0                                           趕稿中，明天晚上見喔   
1    糟糕 以後要說94bb惹 #狂阿編\n\n提名投稿狂新聞>> https://goo.gl/...   
2                                   禮拜一就要來抽獎囉！趕快分享不然掰掰   
3      殺得你措手不及 #狂阿編\n\n提名投稿狂新聞>> https://goo.gl/lrVe2h   
4    百戰百勝 嘿嘿嘿！ #狂阿編\n\n提名投稿狂新聞>> https://goo.gl/lrVe2h   
5       何不食肉糜？ #狂阿編\n\n提名投稿狂新聞>> https://goo.gl/lrVe2h   
6                              小編下班回家收到一封信，內容我就不說了....   
7    【狂抽猛送】第六集 「LINE旅遊大亨」\n\n沒有最狂只有更狂，這次狂抽猛送直接他媽帶你出...   
8      來 這鍋誰扛？ #狂阿編\n\n提名投稿狂新聞>> https://goo.gl/lrVe2h   
9    晚上7點發佈【第6集狂抽猛送】\n　\n一樣保留部分獎項給前一萬樓留言\n而且這一集第一次要...   
10   了不起。盜圖 #狂阿編\n#好圖不盜嗎\n\n提名投稿狂新聞>> https://goo.g...   
11                                 請給這顆石頭一個讚，明天抽獎加碼送石頭   
12   部長您要不要再想想？ #狂阿編\n\n提名投稿狂新聞>> https://goo.gl/lr...   
13   快找議員來幫忙抓一下 #狂阿編\n\n提名投稿狂新聞>> https://goo.gl/lr...   
14   讓點機會給其他人啦 #狂阿編\n\n提名投稿狂新聞>> https://goo.gl/lrVe2h   
15   午飯時間，來跟大家說個可喜可賀的好消息 #小助手\n又有兩台電視被中獎人領走啦~\n\n不用...   
16   【0319】卡提諾狂新聞 #057\n\n● 重複留言絕對無法抽獎！\n　\n※ 第四名由於...   
17                                          20：00 準時更新   
18                                  我知道你們在等，一有消息我就告訴你們   
19   委員 請問公蝦小嗎？ #狂阿編\n\n提名投稿狂新聞>> https://goo.gl/lr...   
20   📣小助手來公佈【狂抽猛送 #5】得獎者名單\n本次獎項真的多到不.手.軟\n如果你是以下得...   
21     議員您這是何苦 #狂阿編\n\n提名投稿狂新聞>> https://goo.gl/lrVe2h   
22   先來公布吃飯不付錢的客人姓名\n阿不是....是分享那則貼文的得獎人姓名 #小助手\n\n【...   
23                             半路